In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [9]:
#multiple batches make training faster
num_epochs = 5
batch_size = 4
learning_rate = 0.001

In [10]:
#tranfrom to have 0 mean and unit variance and making the arrays tensors
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
#loading data and applying the transforms 
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
#the output class NOT ONE HOT ENCODED
classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')

In [11]:
#structure of the CNN
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, 5)#10 output channels one input channel since grayscale image
        self.pool = nn.MaxPool2d(2, 2)#pooling to reduce size with strite 2
        self.conv2 = nn.Conv2d(10, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 256)#first fully connected layer(hidden)
        self.fc2 = nn.Linear(256,128)#second fully connected layer(hidden)
        self.fc3 = nn.Linear(128,10)#output layer

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [12]:
#defining the model
model = ConvNet()
#multiple class output so using CSE loss and SGD optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
#device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()#empty the gradients
        loss.backward()
        optimizer.step()

        if (i+1) % 2000 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

print('Finished Training')

Epoch [1/5], Step [2000/15000], Loss: 2.2586
Epoch [1/5], Step [4000/15000], Loss: 2.2325
Epoch [1/5], Step [6000/15000], Loss: 1.3859
Epoch [1/5], Step [8000/15000], Loss: 0.4952
Epoch [1/5], Step [10000/15000], Loss: 0.7993
Epoch [1/5], Step [12000/15000], Loss: 0.5244
Epoch [1/5], Step [14000/15000], Loss: 0.6856
Epoch [2/5], Step [2000/15000], Loss: 0.0945
Epoch [2/5], Step [4000/15000], Loss: 0.1641
Epoch [2/5], Step [6000/15000], Loss: 0.1333
Epoch [2/5], Step [8000/15000], Loss: 0.0482
Epoch [2/5], Step [10000/15000], Loss: 0.0137
Epoch [2/5], Step [12000/15000], Loss: 0.0329
Epoch [2/5], Step [14000/15000], Loss: 0.2242
Epoch [3/5], Step [2000/15000], Loss: 0.0446
Epoch [3/5], Step [4000/15000], Loss: 0.0959
Epoch [3/5], Step [6000/15000], Loss: 0.0940
Epoch [3/5], Step [8000/15000], Loss: 0.0025
Epoch [3/5], Step [10000/15000], Loss: 0.0513
Epoch [3/5], Step [12000/15000], Loss: 0.0151
Epoch [3/5], Step [14000/15000], Loss: 0.4118
Epoch [4/5], Step [2000/15000], Loss: 0.0003
E

In [19]:
#calculating accuracy
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        #calculating the index of max value
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        #overall accuracy
        n_correct += (predicted == labels).sum().item()
        #classwise accuracy
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if label == pred:
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')


Accuracy of the network: 98.17 %
Accuracy of 0: 99.38775510204081 %
Accuracy of 1: 99.29515418502203 %
Accuracy of 2: 98.83720930232558 %
Accuracy of 3: 98.11881188118812 %
Accuracy of 4: 98.87983706720978 %
Accuracy of 5: 98.87892376681614 %
Accuracy of 6: 96.76409185803757 %
Accuracy of 7: 97.85992217898833 %
Accuracy of 8: 98.4599589322382 %
Accuracy of 9: 95.14370664023787 %
